In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [4]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

## volatility

In [18]:

# Flatten MultiIndex columns 
spy.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in spy.columns
]

# Calculate True Range
spy["prev_close"] = spy["Close_SPY"].shift(1)
spy["tr1"] = spy["High_SPY"] - spy["Low_SPY"]
spy["tr2"] = abs(spy["High_SPY"] - spy["prev_close"])
spy["tr3"] = abs(spy["Low_SPY"] - spy["prev_close"])

spy["true_range"] = spy[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
spy["ATR_10"] = spy["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
spy["atr_pct"] = spy["ATR_10"] / spy["Close_SPY"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

spy["position_size"] = spy.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#spy["allocation_dollars"] = spy["position_size"] * capital

# ---- Preview ----
print(spy[["Close_SPY", "ATR_10", "atr_pct", "position_size"]].tail(10))


             Close_SPY    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  611.091675  4.794563  0.007846           0.01
2025-02-20  608.549377  4.806522  0.007898           0.01
2025-02-21  598.140686  5.513399  0.009218           0.01
2025-02-24  595.418884  5.359863  0.009002           0.01
2025-02-25  592.457764  5.718790  0.009653           0.01
2025-02-26  592.756836  6.146507  0.010369           0.01
2025-02-27  583.295288  6.801538  0.011661           0.01
2025-02-28  592.397949  7.353875  0.012414           0.01
2025-03-03  582.019165  8.901222  0.015294           0.01
2025-03-04  575.129883  9.901217  0.017216           0.01


In [19]:
import plotly.express as px
fig = px.line(spy, x=spy.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [20]:
import plotly.express as px

# Filter only 2025 data
spy_2025 = spy[spy.index.year == 2025]

# Plot
fig = px.line(spy_2025, x=spy_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [36]:
# Without sentiment score
spy_copy = spy.copy()
spy_copy['atr_pct'] = spy_copy['ATR_10'] / spy_copy['Close_SPY']

# Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
spy_copy['Buy_Signal'] = spy_copy['VI+_'] > spy_copy['VI-_']  # Vortex crossover
# + add any other buy conditions here...

# Create Sell Signal (basic)
spy_copy['Sell_Signal'] = spy_copy['VI-_'] > spy_copy['VI+_']

# Initialize position state
spy_copy['Position'] = 0
peak_price = 0

for i in range(1, len(spy_copy)):
    if spy_copy['Buy_Signal'].iloc[i]:
        spy_copy.at[spy_copy.index[i], 'Position'] = 1
        peak_price = spy_copy['Close_SPY'].iloc[i]
    elif spy_copy['Position'].iloc[i - 1] == 1:
        current_price = spy_copy['Close_SPY'].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03:
            spy_copy.at[spy_copy.index[i], 'Sell_Signal'] = True  # trailing stop
            spy_copy.at[spy_copy.index[i], 'Position'] = 0
        else:
            spy_copy.at[spy_copy.index[i], 'Position'] = 1


In [38]:
capital = 100000
in_position = False
entry_price = 0
position_value = 0
cash = capital
returns = []

for i in range(len(spy_copy)):
    row = spy_copy.iloc[i]
    
    # Buy
    if row['Buy_Signal'] and not in_position:
        position_size = row['position_size']
        position_value = cash * position_size
        entry_price = row['Close_SPY']
        shares_bought = position_value / entry_price
        cash -= position_value
        in_position = True
        
    # Sell
    elif row['Sell_Signal'] and in_position:
        exit_price = row['Close_SPY']
        proceeds = shares_bought * exit_price
        profit = proceeds - position_value
        cash += proceeds
        returns.append(profit)
        in_position = False
        position_value = 0
        entry_price = 0

# Final capital
final_value = cash + (shares_bought * row['Close_SPY'] if in_position else 0)
total_return = final_value - capital

print(f"Final Capital: ${final_value:.2f}")
print(f"Total Return: ${total_return:.2f}")
print(f"Total Trades: {len(returns)}")
print(f"Average Profit per Trade: ${np.mean(returns):.2f}")


Final Capital: $100515.03
Total Return: $515.03
Total Trades: 56
Average Profit per Trade: $9.20


In [39]:
import vectorbt as vbt

spy = spy_copy.dropna(subset=['Close_SPY'])
entries = spy_copy['Buy_Signal'].astype(bool)
exits = spy_copy['Sell_Signal'].astype(bool)

price = spy_copy['Close_SPY']
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

print(portfolio.stats())
portfolio.plot().show()



/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/begiii/Documents/SPRING 2025/Hackathon_Spring_2025/Vortex-Sentiment-Adaptive-Volatility-VSAV-Strategy/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           149876.350772
Total Return [%]                        49.876351
Benchmark Return [%]                   153.411688
Max Gross Exposure [%]                      100.0
Total Fees Paid                      14500.396545
Max Drawdown [%]                        19.809401
Max Drawdown Duration                       584.0
Total Trades                                   56
Total Closed Trades                            56
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            55.357143
Best Trade [%]                           7.385098
Worst Trade [%]                         -9.885431
Avg Winning Trade [%]                    3.135411
Avg Losing Trade [%]                    -2.130083
